# ADALA Quickstart

In this notebook, we are going to run through some of the common tasks for creating data labeling agents with ADALA. In this example, we're going to create a data labeling agent for a text classification task - labeling our text samples as either "Subjective or "Objective" statements. 

This agent will be LLM-based, so we will use [OpenAI's API](https://platform.openai.com/). You will to generate an API key and set it as an environment variable as follows: 

```
export OPENAI_API_KEY=your_openai_api_key
```

Now, let's begin. 

## Dataset Creation
First, let's use a dataset of product reviews stored in pandas dataframe. This will help us manage our data as we add more attributes, like predictions and labels for subjectivity and objectivity over time. 

In [3]:
import pandas as pd

df = pd.DataFrame([
    ["The mic is great.", "Subjective"],
    ["Will order from them again!", "Subjective"],
    ["Not loud enough and doesn't turn on like it should.", "Objective"],
    ["The phone doesn't seem to accept anything except CBR mp3s", "Objective"],
    ["All three broke within two months of use.", "Objective"]
], columns=["text", "ground_truth"])

df

,text,ground_truth
0,The mic is great.,Subjective
1,Will order from them again!,Subjective
2,Not loud enough and doesn't turn on like it sh...,Objective
3,The phone doesn't seem to accept anything exce...,Objective
4,All three broke within two months of use.,Objective


We instantiate Dataset that uses this pandas dataframe as a data source. Dataset object takes care of input data schema and data streaming:

In [4]:
from adala.datasets import DataFrameDataset

dataset = DataFrameDataset(df=df)

## Create Agent

To create Agent, we need to to define 2 things:

**Skills** - Agent's abilities are defined as _Skills_. Each agent can possess many different skills. In our case, this agent only has one labeling skill, to produce a classification of Subjective or Objective for a given piece of text.  To define this skill, we will leverage an LLM, passing it instructions and the set of labeles we expect to receive back. 

**Environment** - that is where the Agent receives ground truth signal to improve its skill. Since we already created ground truth dataset, we can simply refer to the column from the dataframe. In the real world scenario, you may consider using a different environment where ground truth signal can be obtained asynchoronously by gathering real human feedback during agent's learning phase.

In [6]:
from adala.agents import Agent
from adala.environments import BasicEnvironment
from adala.skills import ClassificationSkill


agent = Agent(
    # define the agent's labeling skill that should classify text onto 2 categories
    skills=ClassificationSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels=['Subjective', 'Objective']
    ),
    
    # basic environment extracts ground truth signal from the input records
    environment=BasicEnvironment(
        ground_truth_dataset=dataset,
        ground_truth_column='ground_truth'
    ),
)

agent

Agent(environment=BasicEnvironment(ground_truth_dataset=DataFrameDataset(df=                                                text ground_truth
0                                  The mic is great.   Subjective
1                        Will order from them again!   Subjective
2  Not loud enough and doesn't turn on like it sh...    Objective
3  The phone doesn't seem to accept anything exce...    Objective
4          All three broke within two months of use.    Objective), ground_truth_column='ground_truth'), skills=LinearSkillSet(skills={'skill_0': ClassificationSkill(name='subjectivity_detection', instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.', description='Understanding subjective and objective statements from text.', input_template='Input: {{text}}', input_data_field='text', output_template="Output: {{select 'predictions' options=labels logprobs='score'}}", prediction_field='predictions', labels=['Subjective', 'Objective'])}, skill

## Learning Agent

We will now let Agent learn from the ground truth. After every action, Agent returns its _Experience_, where it stores various observations like predicted data, errors, accuracy, etc.

In [8]:
learning_experience = agent.learn(learning_iterations=3, accuracy_threshold=0.95)

Stop program system LLM prompt was created by concatenating instructions with text input:

Prediction = LLM(Input, Instructions)

We expect the prediction to be equal to the ground truth.
Your task is to craft a revised concise instruction for the LLM. Follow best practices for LLM prompt engineering.
Include 2-3 examples at the end of your response to demonstrate how the new instruction would be applied.
Use the following format for your examples:
Input: ...
Output: ... user Old instruction: Determine whether the given product review contains "Subjective" (based on personal feelings, tastes, or opinions) or "Objective" (based on facts or observable phenomena) statements.

Examples:
Input: The camera quality is 12 megapixels.
Output: Objective

Input: I think the camera quality is not good enough.
Output: Subjective

Input: The laptop has a 15-inch screen.
Output: Objective
Errors: 
Input: Input: Not loud enough and doesn't turn on like it should. 
Prediction: Subjective 
Ground truth: Objective 

Input: Input: The phone doesn't seem to accept anything except CBR mp3s 
Prediction: Subjective 
Ground truth: Objective 
 
New instruction: assistant Identify if the provided product review is "Subjective" (based on personal feelings, tastes, or opinions) or "Objective" (based on factual information or observable phenomena). Consider a statement as objective if it presents factual information about the product, even if it describes a problem or issue with the product.

Examples:
Input: The camera quality is 12 megapixels.
Output: Objective

Input: I feel the camera quality is not good enough.
Output: Subjective

Input: The laptop has a 15-inch screen.
Output: Objective

Input: Not loud enough and doesn't turn on like it should.
Output: Objective

Input: The phone doesn't seem to accept anything except CBR mp3s.
Output: Objective

10

Iteration #2: Comparing to ground truth, analyzing and improving...
Accuracy threshold reached (1.0 >= 0.95)
Done!


Let's inspect output predictions:

In [9]:
learning_experience.predictions

,text,ground_truth,subjectivity_detection,score
0,The mic is great.,Subjective,Subjective,"{'Subjective': -0.08425417000000004, 'Objectiv..."
1,Will order from them again!,Subjective,Subjective,"{'Subjective': -0.12256648000000006, 'Objectiv..."
2,Not loud enough and doesn't turn on like it sh...,Objective,Objective,"{'Subjective': -0.7586168000000001, 'Objective..."
3,The phone doesn't seem to accept anything exce...,Objective,Objective,"{'Subjective': -2.7333121, 'Objective': -0.067..."
4,All three broke within two months of use.,Objective,Objective,"{'Subjective': -2.8766944, 'Objective': -0.057..."


## Applying learned skills to the real data

Now as we have our Agent with evolved "subjectivity detection" skill, we can apply it to the real dataset without ground truth data:

In [10]:
test_df = pd.DataFrame([
    "Doesn't hold charge.",
    "Excellent bluetooth headset",
    "I love this thing!",
    "VERY DISAPPOINTED."
], columns=['text'])
test_df

,text
0,Doesn't hold charge.
1,Excellent bluetooth headset
2,I love this thing!
3,VERY DISAPPOINTED.


In [13]:
result = agent.apply_skills(DataFrameDataset(df=test_df))

10


In [14]:
result.predictions

,text,subjectivity_detection,score
0,Doesn't hold charge.,Objective,"{'Subjective': -3.3353982, 'Objective': -0.036..."
1,Excellent bluetooth headset,Subjective,"{'Subjective': -0.16196546999999997, 'Objectiv..."
2,I love this thing!,Subjective,"{'Subjective': -0.003685936600000032, 'Objecti..."
3,VERY DISAPPOINTED.,Subjective,"{'Subjective': -0.07688442600000005, 'Objectiv..."
